In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
%%time

df_raw = pd.read_csv(
    "../data/documenten_ph/DB_V2.TXT",
    encoding="latin_1",
#     nrows=1000,
    sep="\t"
)

In [ ]:
df = df_raw[
    (df_raw.Type == "Général") # focus op niet psychiatrische ziekenhuizen
#     & (df_raw.Province == "Brabant Flamand") # arbitrair regio
    & (df_raw.drug_code != 0) # invalide drug code
#     & (df_raw.hosp_serv_id == 220) # arbitrair 1 departement
    & (df_raw.realization_date % 10 < 5) # uitzondering niet te herleiden tot 1 van 4 quartalen
    & (df_raw.realization_date // 10 > 2002)
]

In [ ]:
min(df_raw.quantity) # negative quantities occur

In [ ]:
df["realization_date_linear"] = df.realization_date // 10 + df.realization_date % 10 / 4
df["realization_date_year"] = df.realization_date // 10
df["realization_date_5_year"] = (df.realization_date // 10 - 3) // 5

## Select the 10 most common drugs

In [ ]:
drug_quantities = df.groupby(["drug_code"]).agg({"quantity": "sum"})

df_common = df[df.drug_code.isin(
    drug_quantities.sort_values(by="quantity", ascending=False).head(10).index
)]

In [ ]:
plt.figure(1, figsize=(16, 16))

ax = plt.subplot("311")
df_evolution = df_common.groupby(
    ["realization_date_linear", "drug_code"]
).agg(
    {"quantity": "sum"}
)

df_evolution.quantity.unstack().plot(marker="o", ax=ax)

ax = plt.subplot("312")
df_evolution = df_common.groupby(
    ["realization_date_year", "drug_code"]
).agg(
    {"quantity": "sum"}
)

df_evolution.quantity.unstack().plot(marker="o", ax=ax)

ax = plt.subplot("313")
df_evolution = df_common.groupby(
    ["realization_date_5_year", "drug_code"]
).agg(
    {"quantity": "sum"}
)

df_evolution.quantity.unstack().plot(marker="o", ax=ax)